In [1]:
import argparse, json, sys, os, shutil
import xarray as xr
import pandas as pd
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.animation import FFMpegWriter
from tqdm import tqdm
import traceback

In [5]:
sys.path.insert(0, '/home/niell_lab/Documents/github/FreelyMovingEphys/')

In [6]:
from project_analysis.jumping.jump_utils import *

In [7]:
config_path = '/home/niell_lab/data/jumping/timebin_eyecams_2sec/jump_analysis_config.json'
with open(config_path, 'r') as fp:
    config = json.load(fp)

In [8]:
pooled_data = xr.open_dataset('/home/niell_lab/data/jumping/timebin_eyecams_2sec/pooled_jump_data.nc')
early_pooled_data = xr.open_dataset('/home/niell_lab/data/jumping/timebin_eyecams_2sec/early_pooled_jump_data.nc')
jumpprep_pooled_data = xr.open_dataset('/home/niell_lab/data/jumping/timebin_eyecams_2sec/jumpprep_pooled_jump_data.nc')
late_pooled_data = xr.open_dataset('/home/niell_lab/data/jumping/timebin_eyecams_2sec/late_pooled_jump_data.nc')

In [9]:
combined_pool_by_animal = pooled_jump_analysis(pooled_data, config, 'combined')
early_pool_by_animal = pooled_jump_analysis(early_pooled_data, config, 'early')
jumpprep_pool_by_animal = pooled_jump_analysis(jumpprep_pooled_data, config, 'jumpprep')
late_pool_by_animal = pooled_jump_analysis(late_pooled_data, config, 'late')

In [11]:
pdf = PdfPages(os.path.join(config['analysis_save_dir'], 'pooled_hists_by_animal.pdf'))

for measurment_name in ['eye_th_div', 'head_pitch']:
    div_early = early_pool_by_animal.sel(jump_params=measurment_name)
    div_jumpprep = jumpprep_pool_by_animal.sel(jump_params=measurment_name)
    early = div_early.drop('variable').rename({'variable':'jump'}).to_array()
    jumpprep = div_jumpprep.drop('variable').rename({'variable':'jump'}).to_array()
    early_bins = jump_hist1(early)
    jumpprep_bins = jump_hist1(jumpprep)
    early = mean_within_animal1(early)
    jumpprep = mean_within_animal1(jumpprep)

    lags = np.arange(-25,25)
    plt.subplots(1,2, figsize=(9,4))
    plt.subplot(1,2,1)
    plt.title('early')
    plt.plot(lags, early_bins.T, alpha=0.3)
    plt.ylabel('proportion of time'); plt.xlabel(measurment_name)
    plt.subplot(1,2,2)
    plt.title('jumpprep')
    plt.ylabel('proportion of time')
    plt.xlabel(measurment_name)
    plt.plot(lags, jumpprep_bins.T, alpha=0.3)
    pdf.savefig()
    plt.close()

    axisplot = 0
    y1 = np.mean(early_bins,axisplot)
    err1 = np.std(np.array(early_bins,dtype=np.float64),axisplot)/np.sqrt(np.size(early_bins,axisplot))
    y2 = np.mean(jumpprep_bins,axisplot)
    err2 = np.std(np.array(jumpprep_bins,dtype=np.float64),axisplot)/np.sqrt(np.size(jumpprep_bins,axisplot))

    lags = np.arange(-25,25)
    plt.figure(figsize=(4,4))
    plt.plot(lags, y1)
    plt.fill_between(lags, y1-err1, y1+err1, alpha=0.3)
    plt.plot(lags, y2)
    plt.fill_between(lags, y2-err2, y2+err2, alpha=0.3)
    plt.ylabel('proportion of time'); plt.xlabel(measurment_name)
    plt.legend(['early', 'jump prep'])
    pdf.savefig()
    plt.close()

pdf.close()